## 1. Data Preprocessing

In [ ]:
# !pip install pandas

In [ ]:
# !pip install seaborn

In [ ]:
# !pip install matplotlib

In [ ]:
# !pip install sklearn

In [ ]:
import pandas as pd  #for data manipulation operations
import numpy as np  #for numeric operations on data
import seaborn as sns  #for data visualization operations
import matplotlib.pyplot as plt  #for data visualization operations
%matplotlib inline

from sklearn.preprocessing import LabelEncoder # for encoding
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler #for standardization
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

import time

In [ ]:
smoking = pd.read_csv("../data/Smoking_raw/smoking.csv")
smoking.head()

In [ ]:
# to show all columns
pd.set_option('display.max_columns', 30)
desc = smoking.describe()

In [ ]:
desc.T[['min','max']]

In [ ]:
smoking.info()

In [ ]:
print("\nThere are totally {} null values in the dataset".format(smoking.isnull().sum().sum())) # for checking null counts

In [ ]:
def preprocessing(df_scaling):
    # 불필요한 컬럼 제거
    if 'ID' in df_scaling:
        df_scaling = df_scaling.drop("ID", axis = 1)
        if len(df_scaling.columns) == 1:
            return df_scaling
        
        # Oral(=구강검사 여부) 특성값은 모두 Y 값이므로 삭제.
        df_scaling = df_scaling.drop('oral', axis = 1) 
        
        # 범주형 피처 레이블 인코딩 
        cate_features = df_scaling[['gender','tartar']]

        lbe = LabelEncoder()
        lbe.fit_transform(df_scaling["gender"])
        df_scaling["gender"] = lbe.fit_transform(df_scaling["gender"])

        lbe = LabelEncoder()
        lbe.fit_transform(df_scaling["tartar"])
        df_scaling["tartar"] = lbe.fit_transform(df_scaling["tartar"])
        
        # hearing 피처 1, 2 => 1, 0으로 변환
        df_scaling['hearing(left)'] = df_scaling['hearing(left)'].apply(lambda x: x-2 if x == 2.0 else x )
        df_scaling['hearing(right)'] = df_scaling['hearing(right)'].apply(lambda x: x-2 if x == 2.0 else x )
        
        # BMI 지수 계산 : bmi = kg/m^2
        df_scaling['bmi'] = df_scaling['weight(kg)']/((df_scaling['height(cm)']*0.01)**2)
        # wwi(비만 지수) 지수 계산 : wwi = cm/sqrt(kg)
        df_scaling['wwi'] = df_scaling['waist(cm)']/(df_scaling['weight(kg)'].apply(np.sqrt))

    return df_scaling

In [ ]:
def scaling(train_data, test_data, scaled_form = 'MinMaxScaler()'):
    # 형태 별 특성 구분
    train_data = preprocessing(train_data)
    test_data = preprocessing(test_data)
    tr_cate_features = train_data[['gender', 'tartar', 'hearing(right)', 'hearing(left)', 'dental caries']]
    tr_scaled_features = train_data.drop(tr_cate_features.columns, axis=1)
    
    ts_cate_features = test_data[['gender', 'tartar', 'hearing(right)', 'hearing(left)', 'dental caries']]
    ts_scaled_features = test_data.drop(ts_cate_features.columns, axis=1)
    
    if scaled_form == 'StandardScaler()':
        # Standard scaler
        scaler = StandardScaler()
        scaler.fit(tr_scaled_features) # 훈련 데이터에 fit() 적용
        
        # 훈련 데이터와 테스트 데이터에 transform()을 통해 변환
        tr_scaled = scaler.transform(tr_scaled_features)
        ts_scaled = scaler.transform(ts_scaled_features)
        
        train_std_scaled = pd.DataFrame(tr_scaled, columns=tr_scaled_features.columns)
        train_std_scaled[tr_cate_features.columns] = tr_cate_features
        
        test_std_scaled = pd.DataFrame(ts_scaled, columns=ts_scaled_features.columns)
        test_std_scaled[ts_cate_features.columns] = ts_cate_features
        
        return train_std_scaled, test_std_scaled
    
    elif scaled_form == 'RobustScaler()':
        # Robust scaler
        scaler = RobustScaler()
        scaler.fit(tr_scaled_features) # 훈련 데이터에 fit() 적용
        
        # 훈련 데이터와 테스트 데이터에 transform()을 통해 변환
        tr_scaled = scaler.transform(tr_scaled_features)
        ts_scaled = scaler.transform(ts_scaled_features)
        
        # 데이터 프레임 형태로 변환
        train_robust_scaled = pd.DataFrame(tr_scaled, columns=tr_scaled_features.columns)
        train_robust_scaled[tr_cate_features.columns] = tr_cate_features
        
        test_robust_scaled = pd.DataFrame(ts_scaled, columns=ts_scaled_features.columns)
        test_robust_scaled[ts_cate_features.columns] = ts_cate_features
        
        return train_robust_scaled, test_robust_scaled
        
    else:
        # MinMax scaler
        scaler = MinMaxScaler()
        scaler.fit(tr_scaled_features) # 훈련 데이터에 fit() 적용
        
        # 훈련 데이터와 테스트 데이터에 transform()을 통해 변환
        tr_scaled = scaler.transform(tr_scaled_features)
        ts_scaled = scaler.transform(ts_scaled_features)
        
        # 데이터 프레임 형태로 변환
        train_mmx_scaled = pd.DataFrame(tr_scaled, columns = tr_scaled_features.columns)
        train_mmx_scaled[tr_cate_features.columns] = tr_cate_features
        
        test_mmx_scaled = pd.DataFrame(ts_scaled,columns = ts_scaled_features.columns)
        test_mmx_scaled[ts_cate_features.columns] = ts_cate_features

        return train_mmx_scaled, test_mmx_scaled

In [ ]:
df = preprocessing(smoking)
df

In [ ]:
df.corr().style.background_gradient(cmap = "magma")

- gender, hemoglobin, height(cm), weight(cm), triglyceride, Gtp, waist(cm), serum creatinine 가 0.2 이상의 상관계수 값을 가짐

### Competition Format

In [ ]:
# competition_format
x_train = pd.read_csv('../data/Smoking_raw/competition_format/x_train.csv')
x_test = pd.read_csv('../data/Smoking_raw/competition_format/x_test.csv')
y_train = pd.read_csv('../data/Smoking_raw/competition_format/y_train.csv')
y_test = pd.read_csv('../data/Smoking_raw/competition_format/y_test.csv')

In [ ]:
x_train = preprocessing(x_train)
x_test = preprocessing(x_test)
y_train = preprocessing(y_train)
y_test = preprocessing(y_test)

In [ ]:
x_train, x_test = scaling(x_train, x_test, 'StandardScaler()')
# x_train, x_test = scaling(x_train, x_test, 'RobustScaler()')
# x_train, x_test = scaling(x_train, x_test)

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
sns.displot(
    data = df, x = "age", hue = "smoking",
    kind = "hist", height = 5, aspect = 1.5,
    palette="ch:rot=-.25,hue=1,light=.50").set(title =  "density relationship between 'age' and 'smoking' variables");


sns.displot(
    data = df, x = "systolic", hue = "smoking",
    kind = "kde", height = 5, aspect = 1.5,
    palette="ch:rot=-.25,hue=1,light=.50").set(title = "density relationship between 'systolic' and 'smoking' variables");


sns.displot(
    data = df, x = "waist(cm)", hue = "smoking",
    kind = "kde", height = 5, aspect = 1.5, multiple="fill",
    palette="ch:rot=-.25,hue=1,light=.50").set(title = "density relationship between 'waist(cm)' and 'smoking' variables");

sns.displot(
    data = df, x = "bmi", hue = "smoking",
    kind = "kde", height = 5, aspect = 1.5,
    palette="ch:rot=-.25,hue=1,light=.50").set(title = "density relationship between 'bmi' and 'smoking' variables");

sns.displot(
    data = df, x = "wwi", hue = "smoking",
    kind = "kde", height = 5, aspect = 1.5,
    palette="ch:rot=-.25,hue=1,light=.50").set(title = "density relationship between 'wwi' and 'smoking' variables");

## 2. XGBoost

In [ ]:
# select features (estimator)
x = df.drop("smoking", axis = 1)

# target (label)
y = df["smoking"]

In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(x, y,
#                                                     test_size = 0.1,
#                                                     shuffle = True,
#                                                     random_state = 1)

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import GridSearchCV
dtrain = xgb.DMatrix(data=x_train,label=y_train,feature_names=x_train.columns)
dtest = xgb.DMatrix(data=x_test,label=y_test,feature_names=x_train.columns)

In [ ]:
params = {'max-depth':3,
          'eta':0.1, # XGBClassifier일 경우 learning_rate 로 쓴다
          'objective':'binary:logistic', #목적함수 : 0 or 1 이므로 이진 로지스틱 사용
          'eval_metric' : 'logloss', # 오류 함수의 평가 성능 지표 : logloss
          'early_stoppings':100, #100회이상 시행시에도 오류가 내려가지않으면 중단
          'silent' : 0,
          'verbosity':0
} # 트리 깊이 최대 3 , 학습률 0.1 , 
num_rounds = 2000

In [ ]:
wlist = [(dtrain,'train'),(dtest,'eval')]
start = time.time()
xgb_model = xgb.train(params = params, dtrain=dtrain,num_boost_round= num_rounds,
                      early_stopping_rounds=100,evals = wlist)
end = time.time()
print("XGB 수행 시간: {0:.1f} 초 ".format(end - start))
pred_probs = xgb_model.predict(dtest)
preds = [1 if x > 0.5 else 0 for x in pred_probs]

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test,pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
# Compute micro-average ROC curve and ROC area
fpr, tpr, _ = roc_curve(y_test.values, pred_probs)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='red',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='blue', lw=lw, linestyle='--')
plt.xlim([-0.02, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
get_clf_eval(y_test, preds, pred_probs)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize=(10,12)) # 축 반환
plot_importance(xgb_model,ax=ax)

#### Grid Search를 이용한 최적 하이퍼 파라미터 탐색

In [ ]:
from sklearn.model_selection import GridSearchCV
# xgb 모델 생성
xgb = XGBClassifier(learning_rate=0.1, 
                    n_estimators=1000,
                    gamma=0, 
                    subsample=0.8, # 각 트리마다의 관측 데이터 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                    colsample_bytree=0.8, # 각 트리마다의 feature 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                    objective= 'binary:logistic', 
                    verbose=10)

# parameter 들을 dictionary 형태로 설정
xgb_params = { 
              'max_depth':range(3,13,3), 
              'min_child_weight':range(1,6,2)
             }

In [ ]:
start = time.time()
# GridSearch를 통해 최적 hyperparameter를 검색
clf = GridSearchCV(xgb,xgb_params,scoring='accuracy',cv=5)
clf.fit(x_train, y_train)
print(clf.best_params_)
pred = clf.predict(x_test)
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,pred)))
end = time.time()
print("XGB 수행 시간: {0:.1f} 초 ".format(end - start))

In [ ]:
# xgb 모델 생성
xgb = XGBClassifier(learning_rate=0.1, 
                    n_estimators=2000,
                    max_depth=3,
                    min_child_weight=3,
                    gamma=0, 
                    subsample=0.8, # 각 트리마다의 관측 데이터 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                    colsample_bytree=0.8, # 각 트리마다의 feature 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                    objective= 'binary:logistic', 
                    verbose=10)

# parameter 들을 dictionary 형태로 설정
xgb_params = { 
              'gamma':[i/10.0 for i in range(0,5)]
             }

In [ ]:
start = time.time()
# GridSearch를 통해 최적 hyperparameter를 검색
clf = GridSearchCV(xgb,xgb_params,scoring='accuracy',cv=5)
clf.fit(x_train, y_train)
print(clf.best_params_)
pred = clf.predict(x_test)
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,pred)))
end = time.time()
print("XGB 수행 시간: {0:.1f} 초 ".format(end - start))

In [ ]:
# xgb 모델 생성
xgb = XGBClassifier(learning_rate=0.1, 
                    n_estimators=2000,
                    max_depth=3,
                    min_child_weight=3,
                    gamma=0, 
                    subsample=0.8, # 각 트리마다의 관측 데이터 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                    colsample_bytree=0.8, # 각 트리마다의 feature 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                    objective= 'binary:logistic', 
                    verbose=10)

# parameter 들을 dictionary 형태로 설정
xgb_params = { 
              'subsample':[i/10.0 for i in range(6,10)],
              'colsample_bytree':[i/10.0 for i in range(6,10)]
             }

In [ ]:
start = time.time()
# GridSearch를 통해 최적 hyperparameter를 검색
clf = GridSearchCV(xgb,xgb_params,scoring='accuracy',cv=5)
clf.fit(x_train, y_train)
print(clf.best_params_)
pred = clf.predict(x_test)
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,pred)))
end = time.time()
print("XGB 수행 시간: {0:.1f} 초 ".format(end - start))

## 3. Light GBM

In [ ]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score
lgbm_wrapper = LGBMClassifier(n_estimators=2000)
evals = [(x_test,y_test)]

start = time.time()
# 학습 : 조기중단 수행(100)
lgbm_wrapper.fit(x_train,y_train,early_stopping_rounds=100,
                eval_metric='logloss',eval_set=evals, verbose=True)

# 예측
preds = lgbm_wrapper.predict(x_test)
pred_probs = lgbm_wrapper.predict_proba(x_test)[:, 1]
end = time.time()
print("LGBM 수행 시간: {0:.1f} 초 ".format(end - start))
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,preds)))

In [ ]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
get_clf_eval(y_test, preds, pred_proba)

In [ ]:
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
# Compute micro-average ROC curve and ROC area
fpr, tpr, _ = roc_curve(y_test.values, pred_probs)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='red',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='blue', lw=lw, linestyle='--')
plt.xlim([-0.02, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# plot_importance( )를 이용하여 feature 중요도 시각화
from lightgbm import plot_importance
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize=(10, 12))
plot_importance(lgbm_wrapper, ax=ax,importance_type='split')

#### Grid Search를 이용한 최적 하이퍼 파라미터 탐색

In [ ]:
from sklearn.model_selection import GridSearchCV
# lgmb 모델 생성
lgb = LGBMClassifier(learning_rate=0.1, 
                       n_estimators=1000,
                       subsample=0.8, # 각 트리마다의 관측 데이터 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                       colsample_bytree=0.8, # 각 트리마다의 feature 샘플링 비율, default = 1, 일반적으로 0.5 ~ 1
                       verbose=10)

# parameter 들을 dictionary 형태로 설정
lgb_params = { 
              'num_leaves':[20,40,60,80,100], #  num_leaves = 2^(max_depth)는 depth-wise tree와 같은 수의 leaves를 가지게 하여, 이보다 작게 설정해야 오버피팅을 줄일 수 있다.
              'max_depth':range(3,13,3), 
              'min_child_samples':range(1,20,5),
              'min_child_weight':range(1,6,2),
            
             }
# _params = {'num_leaves':[20,40,60,80,100], 
#               'min_child_samples':[5,10,15],
#               'max_depth':[-1,5,10,20],
#               'learning_rate':[0.05,0.1,0.2],
#               'reg_alpha':[0,0.01,0.03]}

In [ ]:
start = time.time()
# GridSearch를 통해 최적 hyperparameter를 검색
clf = GridSearchCV(lgb,lgb_params,scoring='accuracy',cv=5)
clf.fit(x_train, y_train)
print(clf.best_params_)
pred = clf.predict(x_test)
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,pred)))
end = time.time()
print("LGBM 수행 시간: {0:.1f} 초 ".format(end - start))

In [ ]:
lgbm_wrapper = LGBMClassifier(n_estimators=2000,max_depth=3,min_child_smaples=1,min_child_weight=5,num_leaves=20,reg_alpha=0)
evals = [(x_test,y_test)]

start = time.time()
# 학습 : 조기중단 수행(100)
lgbm_wrapper.fit(x_train,y_train,early_stopping_rounds=100,
                eval_metric='logloss',eval_set=evals, verbose=True)

# 예측
preds = lgbm_wrapper.predict(x_test)
pred_proba = lgbm_wrapper.predict_proba(x_test)[:, 1]
end = time.time()
print("LGBM 수행 시간: {0:.1f} 초 ".format(end - start))
print('분류 결과 : {0:.1f} '.format(accuracy_score(y_test,preds)))

In [ ]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
get_clf_eval(y_test, preds, pred_proba)

#### LGBM scaler 별 결과
- Standard Scaler : 수행 시간 =  초, 정확도 = 1.0
- Robust Scaler : 수행 시간 =  초, 정확도 = 1.0
- Min Max Scaler : 수행 시간 = 3.9 초, 정확도 = 1.0

#### 각 Scaler 별 최적 하이퍼 파라미터 수치
- Standard Scaler : 
- Robust Scaler : max_depth=10, min_child_smaples=10, num_leaves=40, reg_alpha=0
- Min Max Scaler : 